Data cleaning:
- clean datetime features
- image --> number of image, if nan -->0
- dont need RecipeIngredientQuantities?
- Ingredients: try one-hot encoding with vector summing, or BERT
- Keywords: do the same thing as ingredients
- Category: maybe dont need cuz reflected in keywords
- Rating: classification: 5 classes, regression:?
- remove recipe yield recipe servings
- ONE EMBEDING MODEL: instructions, keywords, ingredients
- description: 0,1


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


### **Imports**

In [ ]:
import os
import re
import pickle
from datetime import datetime
import pandas as pd
import numpy as np

#### Imports needed for embedding models
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.models import Doc2Vec

In [ ]:

#Path to the main directory
folder_path = '/content/gdrive/My Drive/FinalProject_LakshyanaKC_NhatPham/Data/'

# Path to recipes dataset
recipes296k_path = folder_path + "recipes296k_text_processed.pickle"
fulldata_path    = folder_path + "fulldata_processed.pickle"

#Path to save the embedding models
training_path = folder_path + 'train_df.pickle'
testing_path  = folder_path + 'test_df.pickle'

doc2vec_path    = '/content/gdrive/My Drive/FinalProject_LakshyanaKC_NhatPham/Models/embeddings/doc2vec_mergedt_500e.model'
# # doc2vec1_path  = '/content/gdrive/My Drive/Colab Notebooks/CS6120NLP_FinalProject/Models/doc2vec2.model'
# doc2vec296k_path = '/content/gdrive/My Drive/Colab Notebooks/CS6120NLP_FinalProject/Models/doc2vec296k.model'

# Other models
# word2vec1_path = '/content/gdrive/My Drive/Colab Notebooks/CS6120NLP_FinalProject/Models/word2vec.model'
# doc2vec1_path  = '/content/gdrive/My Drive/Colab Notebooks/CS6120NLP_FinalProject/Models/doc2vec1.model'

os.chdir(folder_path)

**Display Options**

In [ ]:
pd.set_option('display.max_colwidth', 2000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### **Helper Methods**

In [ ]:
def now(phrase):
    print(f"[{datetime.now()}]: {phrase}")
    
now("Success!")
def load(filename):
    now("starting")
    if (os.path.exists(filename)):
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        now("ending")
        return data
    else:
        raise FileNotFoundError(f"could not find {filename}")


## saves as pickle object
def save(filename, obj):
    now("Starting")
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)
        print(f"Saved obj Object to {filename}!")
    now("Ending")
    return ''

[2022-05-01 17:03:47.449423]: Success!


#### **Preprocess Text**

In [ ]:
def str_to_list(x):
  if (str(x)[:3]=='c("' and str(x)[-2:]=='")'):
    x = str(x)[3:-2]
    x_list = list(str(x).split('", "')) 
    
  elif (str(x)[0] =='"' and str(x)[-1]=='"'):
    x_list = [str(x)[1:-1]]

  else:
    x_list = ''
  return (x_list)


In [ ]:
def nltk_position_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [ ]:
def get_part_of_speech(word): 
  """
  create a part-of-speech tagging function
  input: a word
  return: the most likely part of speech for that word
  """
  probable_part_of_speech = wordnet.synsets(word)
  pos_counts = Counter()
  pos_counts["n"] = len(  [ item for item in probable_part_of_speech if item.pos()=="n"]  )
  pos_counts["v"] = len(  [ item for item in probable_part_of_speech if item.pos()=="v"]  )
  pos_counts["a"] = len(  [ item for item in probable_part_of_speech if item.pos()=="a"]  )
  pos_counts["r"] = len(  [ item for item in probable_part_of_speech if item.pos()=="r"]  )
  most_likely_part_of_speech = pos_counts.most_common(1)[0][0]
  return most_likely_part_of_speech

"""
Preprocess the text features.
1. Remove unwanted characters,
2. Add sentence start and end tokens,
3. Lowercase text 
"""
def preprocess(x):
  x = str(x)
  # Remove 'c("', '")', 
  x = re.sub(r'c\("','<s> ', x)
  x = re.sub(r'[\(\"\)\n\\]|nan', '', x) # Remove '(,),\n"' signs and 'nan'
  x = re.sub(r'\.+,|\.+', '.', x)  # Replace '.,', "..." with just '.'

  # Add space between alphabet and numeric charecters
  regex = r"(?i)(?<=\d)(?=[a-z°])|(?<=[a-z°])(?=\d)| +"  # "| +" to replace many spaces with one
  x = re.sub(regex, " ", x)

  # Replace "/" symbols between two words with or
  regex = r"([a-zA-Z ])[\/]+(?=[a-zA-Z ])"
  x = re.sub(regex, r'\1 or ', x)

  # Find end of sentence punctuations like "!","?","." (add . if it's followed by space)
  # Add end of sentence with begin and end of sentence tags
  x = re.sub(r'(?<=[.?!])( +|\Z)',' </s> \n <s> ', x) 
  # x = #remove the last <s> tag if text has '.' at the end 

  # Lowercase text
  x = x.lower()

  # Remove punctuations apart from sentence tags and newline
  x = re.sub(r'\-| +\- +', ' ', x) # Replace '/' between alphabets, "-", " - " with just ' '
  x = re.sub(r'[^\w\s\</s>\<s>\\n]+', '', x) #not including apostrophe \'

  # Add end of sentence token in the last sentence if not present
  x = x + ' </s>' if x[-4:] != '</s>' and x!= ''  else x
  # Add start of sentence token in the first sentence if not present
  x = '<s> '+ x if x[0:3] != '<s>' and x!= ''  else x

  # Remove blank sentences <s> spaces </s> and 'nan'
  x = re.sub(r"\<s> +\</s>", '', x) 

  # 2. Extract tokens
  # tokens = nltk.word_tokenize(x)
  sentences = [sentence.split() for sentence in x.split('\n')]

  # 4. Lemmatize
  # nltk_tagged = nltk.pos_tag(tokens) 
  # wordnet_tagged = map(lambda x: (x[0], nltk_position_tagger(x[1])), nltk_tagged)
  # tokens = [[lemma.lemmatize(word, tag) if tag != None else word] for word,tag in wordnet_tagged]
  sentences = [[lemma.lemmatize(word, get_part_of_speech(word)) for word in tokens] for tokens in sentences]

  # 3. Remove stopword
  tokens = [[word for word in tokens if not word in stopwords] for tokens in sentences if tokens!= []]

  # print("\nUpdated text")
  # print(x)

  # print("\nSentences")
  # print(sentences)

  # print("\nTokens")
  # print(tokens)
  
  return tokens


In [ ]:
"""
Preprocess the text features.
1. Remove unwanted characters,
2. Add sentence start and end tokens,
3. Lowercase text 
"""
def preprocessList(x):
  x = str(x)
  # Remove 'c("', '")', 
  x = re.sub(r'c\(|\)','', x)
  x = re.sub(r'\-|\/', ' ', x) # Replace '/' between alphabets, "-", " - " with just ' '
  x = re.sub(r'[\n\\]|nan', '', x) # Remove '(,),\n"' signs and 'nan'

  # Add space between alphabet and numeric charecters
  # regex = r"(?i)(?<=\d)(?=[a-z°])|(?<=[a-z°])(?=\d)| +"  # "| +" to replace many spaces with one
  # x = re.sub(regex, " ", x)

  # Replace "/" symbols between two words with or
  regex = r"([a-zA-Z ])[\/]+(?=[a-zA-Z ])"
  x = re.sub(regex, r'\1 or ', x)

  # Find end of sentence punctuations like "!","?","." (add . if it's followed by space)
  # Add end of sentence with begin and end of sentence tags
  # x = re.sub(r'(?<=[.?!])( +|\Z)',' </s> \n <s> ', x) 
  # x = #remove the last <s> tag if text has '.' at the end 

  # Lowercase text
  x = x.lower()

  # Remove punctuations apart from sentence tags and newline
  x = re.sub(r'[^\w\s\"]+', '', x) #not including apostrophe \'
  # Replace multiple spaces with one
  x = re.sub(r' +', ' ', x) #not including apostrophe

  # Replace " " between ingredients with "," 
  x = re.sub(r'\" +"', ',', x) 
  # Replace space between each ingredient with "-"
  x = re.sub(r' ', '-', x) #not including apostrophe

  # Now replace "," back to " "
  x = re.sub(r'\,',' ', x) 

  # Add end of sentence token in the last sentence if not present
  # x = x + ' </s>' if x[-4:] != '</s>' and x!= ''  else x
  # Add start of sentence token in the first sentence if not present
  # x = '<s> '+ x if x[0:3] != '<s>' and x!= ''  else x

  # Remove blank sentences <s> spaces </s> and 'nan'
  # x = re.sub(r"\<s> +\</s>", '', x) 

  # 2. Extract tokens
  # tokens = nltk.word_tokenize(x)
  tokens = x.split()

  # # 3. Remove stopword
  # tokens = [[word for word in tokens if not word in stopwords] for tokens in sentences]

  # # 4. Lemmatize
  # tokens = [[lemma.lemmatize(word, get_part_of_speech(word)) for word in tokens] for tokens in sentences]

  # print("\nUpdated text")
  # print(x)

  # print("\nSentences")
  # print(x)

  # print("\nTokens")
  # print(tokens)
  
  return tokens





### **Load Data**

In [ ]:
## Load training data
training_data = load(training_path)

## Load testing data
testing_data  = load(testing_path)

### Load the full dataset
all_recipes = load(fulldata_path)

# reset index of the dataframe to keep the indices ordered consecutively
all_recipes.reset_index(inplace=True, drop='index')
# now reset again to add the new indices as a separate column as well
all_recipes.reset_index(inplace=True)

[2022-05-01 17:03:50.521935]: starting
[2022-05-01 17:03:53.220170]: ending
[2022-05-01 17:03:53.220544]: starting
[2022-05-01 17:03:54.314919]: ending
[2022-05-01 17:03:54.315768]: starting
[2022-05-01 17:03:57.535552]: ending


#### **Understanding the dataset**

In [ ]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201970 entries, 391926 to 94498
Data columns (total 24 columns):
 #   Column                      Non-Null Count   Dtype   
---  ------                      --------------   -----   
 0   RecipeId                    201970 non-null  int64   
 1   Name                        201970 non-null  object  
 2   CookTime                    201970 non-null  float64 
 3   PrepTime                    201970 non-null  float64 
 4   TotalTime                   201970 non-null  float64 
 5   Description                 201970 non-null  int64   
 6   Images                      201970 non-null  int64   
 7   RecipeCategory              201794 non-null  object  
 8   Keywords                    201970 non-null  object  
 9   RecipeIngredientQuantities  201969 non-null  object  
 10  RecipeIngredientParts       201970 non-null  object  
 11  AggregatedRating            201970 non-null  float64 
 12  ReviewCount                 201970 non-null  float64 


In [ ]:
testing_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67324 entries, 57910 to 494908
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   RecipeId                    67324 non-null  int64   
 1   Name                        67324 non-null  object  
 2   CookTime                    67324 non-null  float64 
 3   PrepTime                    67324 non-null  float64 
 4   TotalTime                   67324 non-null  float64 
 5   Description                 67324 non-null  int64   
 6   Images                      67324 non-null  int64   
 7   RecipeCategory              67255 non-null  object  
 8   Keywords                    67324 non-null  object  
 9   RecipeIngredientQuantities  67323 non-null  object  
 10  RecipeIngredientParts       67324 non-null  object  
 11  AggregatedRating            67324 non-null  float64 
 12  ReviewCount                 67324 non-null  float64 
 13  Calories   

In [ ]:
all_recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269294 entries, 0 to 269293
Data columns (total 25 columns):
 #   Column                      Non-Null Count   Dtype   
---  ------                      --------------   -----   
 0   index                       269294 non-null  int64   
 1   RecipeId                    269294 non-null  int64   
 2   Name                        269294 non-null  object  
 3   CookTime                    269294 non-null  float64 
 4   PrepTime                    269294 non-null  float64 
 5   TotalTime                   269294 non-null  float64 
 6   Description                 269294 non-null  int64   
 7   Images                      269294 non-null  int64   
 8   RecipeCategory              269049 non-null  object  
 9   Keywords                    269294 non-null  object  
 10  RecipeIngredientQuantities  269292 non-null  object  
 11  RecipeIngredientParts       269294 non-null  object  
 12  AggregatedRating            269294 non-null  float64 
 13 

In [ ]:
training_data.head(1)

,RecipeId,Name,CookTime,PrepTime,TotalTime,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeInstructions,RatingId
391926,406189,Dulce De Leche Rice Pudding,35.0,5.0,40.0,0,0,Dessert,"[rice, sweet, -60-mins, stove-top, easy]","c(""8"", ""2"", ""1"", ""1"")","[milk, cinnamon-sticks, ground-cinnamon, long-grain-rice]",5.0,1.0,320.8,12.1,7.5,45.5,161.0,39.8,0.4,0.0,12.9,<s> heat milk cinnamon stick large saucepan medium high heat reach boil remove cinnamon stick </s> <s> reduce heat medium low add rice </s> <s> cook stir frequently 15 20 minute </s> <s> add dulce de leche cook stir frequently 15 20 minute rice tender sauce thicken </s> <s> garnish slice fresh fruit desire </s>,5


In [ ]:
testing_data.head(1)

,RecipeId,Name,CookTime,PrepTime,TotalTime,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeInstructions,RatingId
57910,62218,Tasty Twist on Chicken Salad,0.0,10.0,10.0,1,0,Lunch/Snacks,"[chicken, poultry, meat, brunch, -15-mins, beginner-cook, easy]","c(""1"", ""1/4 - 1/2"", ""5 -6"", ""1 -2"", ""2 -4"", NA)","[boneless-skinless-chicken-breast, celery, jicama, green-onion, mayonnaise]",5.0,2.0,250.7,11.3,1.8,76.1,296.1,8.4,0.6,2.4,27.9,<s> add ingredient bowl stir well </s> <s> enjoy like mine straight bowl bread </s>,5


In [ ]:
all_recipes.head(1)

,index,RecipeId,Name,CookTime,PrepTime,TotalTime,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeInstructions,RatingId
0,0,38,Low-Fat Berry Blue Frozen Dessert,1440.0,45.0,1485.0,0,5,Frozen Desserts,"[dessert, low-protein, low-cholesterol, healthy, free-of, summer, weeknight, freezer, easy]","c(""4"", ""1/4"", ""1"", ""1"")","[blueberries, granulated-sugar, vanilla-yogurt, lemon-juice]",4.5,4.0,170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,<s> toss 2 cup berry sugar </s> <s> let stand 45 minute stir occasionally </s> <s> transfer berry sugar mixture food processor </s> <s> add yogurt process smooth </s> <s> strain fine sieve </s> <s> pour bake pan transfer ice cream maker process accord manufacturer direction </s> <s> freeze uncover edge solid centre soft </s> <s> transfer processor blend smooth </s> <s> return pan freeze edge solid </s> <s> transfer processor blend smooth </s> <s> fold remain 2 cup blueberry </s> <s> pour plastic mold freeze overnight </s> <s> let soften slightly serve </s>,4


#### **Prepare Data for Training**

In [ ]:
### training data with the document id
training_data = training_data.merge(all_recipes[['RecipeId','index']], how='inner', left_on='RecipeId', right_on='RecipeId')

### testing data with the document id
testing_data = testing_data.merge(all_recipes[['RecipeId','index']], how='inner', left_on='RecipeId', right_on='RecipeId')

##### **X-train, X-test, Y_train, Y_test**

In [ ]:
def merge_list(input_list):
  return " ".join(input_list)

def merge_list(input_list):
  return " ".join(input_list)
  
# Full dataset
data = all_recipes
text_feature = data[["RecipeId","Name","RecipeCategory",'RatingId','RecipeIngredientParts','RecipeInstructions']]
text_feature.reset_index(inplace=True, drop='index')
text_feature.reset_index(inplace=True)

text_feature['RecipeIngredientParts'] = text_feature['RecipeIngredientParts'].apply(merge_list)
text_feature['Recipe'] = text_feature['RecipeIngredientParts'].astype(str) + " "+ text_feature['RecipeInstructions']

# text_feature = text_feature[text_feature['RecipeInstructions'].apply(lambda x: len(str(x))<90)] 
# text_feature =  text_feature[text_feature['RatingId'].apply(lambda x: x<4)]
# text_feature = text_feature[text_feature['RecipeInstructions'].apply(lambda x: len(str(x))<200 and len(str(x)) > 100)]
# display(text_feature)

# Training data
data = training_data
training_data = data[["index","RecipeId","Name","RecipeCategory",'RatingId','RecipeIngredientParts','RecipeInstructions']]
training_data['RecipeIngredientParts'] = training_data['RecipeIngredientParts'].apply(merge_list)
training_data['Recipe'] = training_data['RecipeIngredientParts'].astype(str) + " "+ training_data['RecipeInstructions']

# Testing data
data = testing_data
testing_data = data[["index","RecipeId","Name","RecipeCategory",'RatingId','RecipeIngredientParts','RecipeInstructions']]
testing_data['RecipeIngredientParts'] = testing_data['RecipeIngredientParts'].apply(merge_list)
testing_data['Recipe'] = testing_data['RecipeIngredientParts'].astype(str) + " "+ testing_data['RecipeInstructions']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [ ]:
print(len(text_feature))
text_feature.head(1)

269294


,index,RecipeId,Name,RecipeCategory,RatingId,RecipeIngredientParts,RecipeInstructions,Recipe
0,0,38,Low-Fat Berry Blue Frozen Dessert,Frozen Desserts,4,blueberries granulated-sugar vanilla-yogurt lemon-juice,<s> toss 2 cup berry sugar </s> <s> let stand 45 minute stir occasionally </s> <s> transfer berry sugar mixture food processor </s> <s> add yogurt process smooth </s> <s> strain fine sieve </s> <s> pour bake pan transfer ice cream maker process accord manufacturer direction </s> <s> freeze uncover edge solid centre soft </s> <s> transfer processor blend smooth </s> <s> return pan freeze edge solid </s> <s> transfer processor blend smooth </s> <s> fold remain 2 cup blueberry </s> <s> pour plastic mold freeze overnight </s> <s> let soften slightly serve </s>,blueberries granulated-sugar vanilla-yogurt lemon-juice <s> toss 2 cup berry sugar </s> <s> let stand 45 minute stir occasionally </s> <s> transfer berry sugar mixture food processor </s> <s> add yogurt process smooth </s> <s> strain fine sieve </s> <s> pour bake pan transfer ice cream maker process accord manufacturer direction </s> <s> freeze uncover edge solid centre soft </s> <s> transfer processor blend smooth </s> <s> return pan freeze edge solid </s> <s> transfer processor blend smooth </s> <s> fold remain 2 cup blueberry </s> <s> pour plastic mold freeze overnight </s> <s> let soften slightly serve </s>


In [ ]:
print(len(training_data))
training_data.head(1)

201970


,index,RecipeId,Name,RecipeCategory,RatingId,RecipeIngredientParts,RecipeInstructions,Recipe
0,235422,406189,Dulce De Leche Rice Pudding,Dessert,5,milk cinnamon-sticks ground-cinnamon long-grain-rice,<s> heat milk cinnamon stick large saucepan medium high heat reach boil remove cinnamon stick </s> <s> reduce heat medium low add rice </s> <s> cook stir frequently 15 20 minute </s> <s> add dulce de leche cook stir frequently 15 20 minute rice tender sauce thicken </s> <s> garnish slice fresh fruit desire </s>,milk cinnamon-sticks ground-cinnamon long-grain-rice <s> heat milk cinnamon stick large saucepan medium high heat reach boil remove cinnamon stick </s> <s> reduce heat medium low add rice </s> <s> cook stir frequently 15 20 minute </s> <s> add dulce de leche cook stir frequently 15 20 minute rice tender sauce thicken </s> <s> garnish slice fresh fruit desire </s>


In [ ]:
print(len(testing_data))
testing_data.head(1)

67324


,index,RecipeId,Name,RecipeCategory,RatingId,RecipeIngredientParts,RecipeInstructions,Recipe
0,44736,62218,Tasty Twist on Chicken Salad,Lunch/Snacks,5,boneless-skinless-chicken-breast celery jicama green-onion mayonnaise,<s> add ingredient bowl stir well </s> <s> enjoy like mine straight bowl bread </s>,boneless-skinless-chicken-breast celery jicama green-onion mayonnaise <s> add ingredient bowl stir well </s> <s> enjoy like mine straight bowl bread </s>


In [ ]:
training_data.head(1)

,index,RecipeId,Name,RecipeCategory,RatingId,RecipeIngredientParts,RecipeInstructions,Recipe
0,235422,406189,Dulce De Leche Rice Pudding,Dessert,5,milk cinnamon-sticks ground-cinnamon long-grain-rice,<s> heat milk cinnamon stick large saucepan medium high heat reach boil remove cinnamon stick </s> <s> reduce heat medium low add rice </s> <s> cook stir frequently 15 20 minute </s> <s> add dulce de leche cook stir frequently 15 20 minute rice tender sauce thicken </s> <s> garnish slice fresh fruit desire </s>,milk cinnamon-sticks ground-cinnamon long-grain-rice <s> heat milk cinnamon stick large saucepan medium high heat reach boil remove cinnamon stick </s> <s> reduce heat medium low add rice </s> <s> cook stir frequently 15 20 minute </s> <s> add dulce de leche cook stir frequently 15 20 minute rice tender sauce thicken </s> <s> garnish slice fresh fruit desire </s>


#### **Train Embedding Models**

##### **1.1 Prepare Dataset for training word2vec model**

In [ ]:
data = text_feature['Recipe'].tolist()

# print(data[10])
# data = [sentence for row in data for sentence in row]

##### **1.2 Train a word2vec model**

In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

import multiprocessing
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

EMBEDDINGS_SIZE = 100
model = Word2Vec(sentences= data, size = EMBEDDINGS_SIZE, window = 10, min_count = 1, 
                  sg = 1, workers = cores-1)


In [ ]:
model.save(word2vec1_path)

In [ ]:
# if you save your Word2Vec as the variable model, this will 
# print out the vocabulary size
print('Vocab size {}'.format(len(model.wv.vocab)))

Vocab size 58758


##### **2.1 Prepare Tagged Documents for Doc2Vec Model**

In [ ]:
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.models import Doc2Vec

d = text_feature['Recipe'].tolist()

## Approach 1
data1 = [gensim.models.doc2vec.TaggedDocument(
             words=[document],
             tags=[i])
         for i, document in enumerate(d)]

### Approach 2
data2 = [gensim.models.doc2vec.TaggedDocument(
             document.split(),
             tags=[i]) 
             for i, document in enumerate(d)]

print(data1[:5])
print(data2[:5])

[TaggedDocument(words=['blueberries granulated-sugar vanilla-yogurt lemon-juice <s> toss 2 cup berry sugar </s> <s> let stand 45 minute stir occasionally </s> <s> transfer berry sugar mixture food processor </s> <s> add yogurt process smooth </s> <s> strain fine sieve </s> <s> pour bake pan transfer ice cream maker process accord manufacturer direction </s> <s> freeze uncover edge solid centre soft </s> <s> transfer processor blend smooth </s> <s> return pan freeze edge solid </s> <s> transfer processor blend smooth </s> <s> fold remain 2 cup blueberry </s> <s> pour plastic mold freeze overnight </s> <s> let soften slightly serve </s> '], tags=[0]), TaggedDocument(words=['saffron milk hot-green-chili-peppers onions garlic clove peppercorns cardamom-seed cumin-seed poppy-seed mace cilantro mint-leaf fresh-lemon-juice plain-yogurt boneless-chicken salt ghee onion tomatoes basmati-rice long-grain-rice raisins cashews eggs <s> soak saffron warm milk 5 minute puree blender </s> <s> add chil

##### **2.2 Train a doc2vec model**

In [ ]:
# build the model
import multiprocessing
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

############ Doc2Vec Model 1 ###############

# PV-DBOW (Skip-Gram equivalent of Word2Vec)
doc2vec_model  = gensim.models.Doc2Vec(dm = 0, alpha=0.025, vector_size=50, min_alpha=0.01, min_count=0)
doc2vec_model.build_vocab(data2)

doc2vec_model.train(data2,total_examples=doc2vec_model.corpus_count,epochs=1000)

#path to save the doc2vec model
save_path = folder_path+'Models/embeddings/doc2vec_mergedt_1000.model'
doc2vec_model.save(save_path)

In [ ]:
import multiprocessing
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

#train model 
doc2vec_model  = gensim.models.Doc2Vec(dm = 0, alpha=0.025, vector_size=100, min_count=0, min_alpha=0.01, workers = cores-1)
doc2vec_model.build_vocab(data1)

doc2vec_model.train(data1, total_examples=doc2vec_model.corpus_count,epochs=1000)
doc2vec_model.save('/content/gdrive/My Drive/FinalProject_LakshyanaKC_NhatPham/Models/doc2vec_merged_1000e.model')

In [ ]:
print("done")

#### **Load the trained doc2vec model**

In [ ]:
###### Model 1
doc2vec_model   = Doc2Vec.load(doc2vec_path)

# ###### Model 2
# doc2vec_model2   = Doc2Vec.load(doc2vec296k_path)

In [ ]:
print(data1[:10])
print(data2[:10])

[TaggedDocument(words=['<s> toss 2 cup berry sugar </s> <s> let stand 45 minute stir occasionally </s> <s> transfer berry sugar mixture food processor </s> <s> add yogurt process smooth </s> <s> strain fine sieve </s> <s> pour bake pan transfer ice cream maker process accord manufacturer direction </s> <s> freeze uncover edge solid centre soft </s> <s> transfer processor blend smooth </s> <s> return pan freeze edge solid </s> <s> transfer processor blend smooth </s> <s> fold remain 2 cup blueberry </s> <s> pour plastic mold freeze overnight </s> <s> let soften slightly serve </s> '], tags=[0]), TaggedDocument(words=['<s> soak saffron warm milk 5 minute puree blender </s> <s> add chile onion ginger garlic clove peppercorn cardamom seed cinnamon coriander cumin seed poppy seed nutmeg mace cilantro mint leave lemon juice </s> <s> blend smooth paste </s> <s> put paste large bowl add yogurt mix well </s> <s> marinate chicken yogurt mixture salt cover least 2 6 hour refrigerator </s> <s> ski

In [ ]:
print(len(data1))
print(len(data2))

269294
269294


#### **Find similar documents**

**Doc2Vec Model 1:**

In [ ]:
# doc2vec_model1.train(data1, total_examples=doc2vec_model1.corpus_count,epochs=1000)
# doc2vec_model1.save('/content/gdrive/My Drive/Colab Notebooks/CS6120NLP_FinalProject/Models/doc2vec_merged_1000e.model')

In [ ]:

doc2vec_model   = Doc2Vec.load(doc2vec_path)
# doc2vec model 1
index = 0
example = text_feature['Recipe'][index]

print(example)
inferred_vector1 = doc2vec_model.infer_vector(example)
sims1  = doc2vec_model.docvecs.most_similar([inferred_vector1])

# print(inferred_vector)
print(sims1)

print("\nDocument")
print(example)
print("Ingredients")
print(text_feature.iloc[[index]])
# print("keywords",text_feature['Keywords'][2])

print("\nSimilar Documents")
for i in range(len(sims1)):
  # print("\nKeywords", text_feature['Keywords'][sims1[i][0]])
  print("Similarity: ", sims1[i][1]) #d[sims1[i][0]]
  # print(sims1[i][0])
  print("Recipe: ")
  display(text_feature.iloc[[sims1[i][0]]])

## Doc 1 seems most similar to doc ..?
# print(' '.join(data[sims2[0][0]])+'\n')

blueberries granulated-sugar vanilla-yogurt lemon-juice <s> toss 2 cup berry sugar </s> <s> let stand 45 minute stir occasionally </s> <s> transfer berry sugar mixture food processor </s> <s> add yogurt process smooth </s> <s> strain fine sieve </s> <s> pour bake pan transfer ice cream maker process accord manufacturer direction </s> <s> freeze uncover edge solid centre soft </s> <s> transfer processor blend smooth </s> <s> return pan freeze edge solid </s> <s> transfer processor blend smooth </s> <s> fold remain 2 cup blueberry </s> <s> pour plastic mold freeze overnight </s> <s> let soften slightly serve </s> 
[(120250, 0.9978649616241455), (124815, 0.6493815779685974), (19981, 0.6439335942268372), (196241, 0.6435356140136719), (212106, 0.6411705017089844), (27712, 0.6361868381500244), (141108, 0.6343872547149658), (119038, 0.633597195148468), (181395, 0.6313998699188232), (91946, 0.6252375245094299)]

Document
blueberries granulated-sugar vanilla-yogurt lemon-juice <s> toss 2 cup be

,index,RecipeId,Name,RecipeCategory,RatingId,RecipeIngredientParts,RecipeInstructions,Recipe
120250,120250,176767,All Season Bread,Quick Breads,4,flour baking-soda salt baking-powder cinnamon eggs white-sugar real-vanilla raw-carrots raisins walnuts,,flour baking-soda salt baking-powder cinnamon eggs white-sugar real-vanilla raw-carrots raisins walnuts


Similarity:  0.6493815779685974
Recipe: 


,index,RecipeId,Name,RecipeCategory,RatingId,RecipeIngredientParts,RecipeInstructions,Recipe
124815,124815,184451,Pat's Coney Sauce No. 15,Sauces,5,green-bell-pepper ketchup prepared-yellow-mustard chili-powder worcestershire-sauce cider-vinegar allspice ground-cumin crushed-red-pepper-flakes ground-cloves,<s> brown burger sausage cover pot </s> <s> break stir </s> <s> drain grease discard </s> <s> return drain burger sausage blend cook pot add ingredient except grape jelly elect use </s> <s> bring boil immediately reduce heat simmer cover 90 minute stir frequently </s> <s> add grape jelly optional last five minute cook stir melt </s> <s> need thin point add little additional chicken stock </s> <s> serve hot dog tangy mustard finely dice onion steam hot dog bun </s> <s> note use cheap grade hot dog grind one include chicken pork </s> <s> beef frank impart much flavor </s> <s> also prefer sauce without jelly personal taste issue decide last minute since jelly get add split sauce half use one tablespoon jelly one half </s>,green-bell-pepper ketchup prepared-yellow-mustard chili-powder worcestershire-sauce cider-vinegar allspice ground-cumin crushed-red-pepper-flakes ground-cloves <s> brown burger sausage cover pot </s> <s> break stir </s> <s> drain grease discard </s> <s> return drain burger sausage blend cook pot add ingredient except grape jelly elect use </s> <s> bring boil immediately reduce heat simmer cover 90 minute stir frequently </s> <s> add grape jelly optional last five minute cook stir melt </s> <s> need thin point add little additional chicken stock </s> <s> serve hot dog tangy mustard finely dice onion steam hot dog bun </s> <s> note use cheap grade hot dog grind one include chicken pork </s> <s> beef frank impart much flavor </s> <s> also prefer sauce without jelly personal taste issue decide last minute since jelly get add split sauce half use one tablespoon jelly one half </s>


Similarity:  0.6439335942268372
Recipe: 


...

,index,RecipeId,Name,RecipeCategory,RatingId,RecipeIngredientParts,RecipeInstructions,Recipe
119038,119038,174672,Crispy Soft Shell Crabs,< 60 Mins,5,soft-shelled-crabs milk all-purpose-flour ground-ginger paprika garlic-powder salted-butter mayonnaise,<s> soak crab milk 30 minute </s> <s> mix flour salt pepper ginger paprika garlic powder bag dredge crab mixture </s> <s> melt butter large skillet medium heat </s> <s> saute crab golden brown 5 minute per side </s> <s> spread bun mayonnaise top crab </s> <s> dollop coleslaw desire </s> <s> serve immediately </s>,soft-shelled-crabs milk all-purpose-flour ground-ginger paprika garlic-powder salted-butter mayonnaise <s> soak crab milk 30 minute </s> <s> mix flour salt pepper ginger paprika garlic powder bag dredge crab mixture </s> <s> melt butter large skillet medium heat </s> <s> saute crab golden brown 5 minute per side </s> <s> spread bun mayonnaise top crab </s> <s> dollop coleslaw desire </s> <s> serve immediately </s>


Similarity:  0.6313998699188232
Recipe: 


,index,RecipeId,Name,RecipeCategory,RatingId,RecipeIngredientParts,RecipeInstructions,Recipe
181395,181395,286956,Pepperoni Pizza Chili,Beans,4,ground-beef kidney-beans stewed-tomatoes tomato-sauce water green-pepper salt mozzarella-cheese,<s> cook grind beef </s> <s> mix ingredient slow cooker </s> <s> cook 8 hour </s> <s> garnish mozzarella cheese desire </s>,ground-beef kidney-beans stewed-tomatoes tomato-sauce water green-pepper salt mozzarella-cheese <s> cook grind beef </s> <s> mix ingredient slow cooker </s> <s> cook 8 hour </s> <s> garnish mozzarella cheese desire </s>


Similarity:  0.6252375245094299
Recipe: 


,index,RecipeId,Name,RecipeCategory,RatingId,RecipeIngredientParts,RecipeInstructions,Recipe
91946,91946,130586,Canned Fresh Tomato Spaghetti Sauce,Sauces,5,fresh-tomato garlic-cloves onions tomato-paste sugar salt oregano,<s> cut tomato run food processor food mill put large pan cook </s> <s> meanwhile cut garlic onion cook oil separate pan run food mill </s> <s> add onion garlic remain ingredient tomato slowly heat boil </s> <s> boil stir occasionally thick 2 hour </s> <s> pour clean hot quart jar leave 1/2 inch head space </s> <s> process 40 minute boil water bath </s>,fresh-tomato garlic-cloves onions tomato-paste sugar salt oregano <s> cut tomato run food processor food mill put large pan cook </s> <s> meanwhile cut garlic onion cook oil separate pan run food mill </s> <s> add onion garlic remain ingredient tomato slowly heat boil </s> <s> boil stir occasionally thick 2 hour </s> <s> pour clean hot quart jar leave 1/2 inch head space </s> <s> process 40 minute boil water bath </s>


In [ ]:
doc2vec_model   = Doc2Vec.load('/content/gdrive/My Drive/Colab Notebooks/CS6120NLP_FinalProject/Models/embeddings/doc2vec_mergedp_500e.model')
# doc2vec model 1
index = 0
example = text_feature['Recipe'][index]

print(example)
inferred_vector1 = doc2vec_model.infer_vector(example)
sims1  = doc2vec_model.docvecs.most_similar([inferred_vector1])

# print(inferred_vector)
print(sims1)

print("\nDocument")
print(example)
print("Ingredients")
print(text_feature.iloc[[index]])
# print("keywords",text_feature['Keywords'][2])

print("\nSimilar Documents")
for i in range(len(sims1)):
  # print("\nKeywords", text_feature['Keywords'][sims1[i][0]])
  print("Similarity: ", sims1[i][1]) #d[sims1[i][0]]
  # print(sims1[i][0])
  print("Recipe: ")
  display(text_feature.iloc[[sims1[i][0]]])
  
## Doc 1 seems most similar to doc ..?
# print(' '.join(data[sims2[0][0]])+'\n')

blueberries granulated-sugar vanilla-yogurt lemon-juice <s> toss 2 cup berry sugar </s> <s> let stand 45 minute stir occasionally </s> <s> transfer berry sugar mixture food processor </s> <s> add yogurt process smooth </s> <s> strain fine sieve </s> <s> pour bake pan transfer ice cream maker process accord manufacturer direction </s> <s> freeze uncover edge solid centre soft </s> <s> transfer processor blend smooth </s> <s> return pan freeze edge solid </s> <s> transfer processor blend smooth </s> <s> fold remain 2 cup blueberry </s> <s> pour plastic mold freeze overnight </s> <s> let soften slightly serve </s> 
[(120250, 0.9960542917251587), (124815, 0.6452643871307373), (19981, 0.6377310752868652), (196241, 0.6323022246360779), (212106, 0.629898726940155), (141108, 0.6270086765289307), (27712, 0.623745322227478), (119038, 0.6222140789031982), (181395, 0.618255078792572), (201372, 0.6178948879241943)]

Document
blueberries granulated-sugar vanilla-yogurt lemon-juice <s> toss 2 cup ber

,index,RecipeId,Name,RecipeCategory,RatingId,RecipeIngredientParts,RecipeInstructions,Recipe
120250,120250,176767,All Season Bread,Quick Breads,4,flour baking-soda salt baking-powder cinnamon eggs white-sugar real-vanilla raw-carrots raisins walnuts,,flour baking-soda salt baking-powder cinnamon eggs white-sugar real-vanilla raw-carrots raisins walnuts


Similarity:  0.6452643871307373
Recipe: 


,index,RecipeId,Name,RecipeCategory,RatingId,RecipeIngredientParts,RecipeInstructions,Recipe
124815,124815,184451,Pat's Coney Sauce No. 15,Sauces,5,green-bell-pepper ketchup prepared-yellow-mustard chili-powder worcestershire-sauce cider-vinegar allspice ground-cumin crushed-red-pepper-flakes ground-cloves,<s> brown burger sausage cover pot </s> <s> break stir </s> <s> drain grease discard </s> <s> return drain burger sausage blend cook pot add ingredient except grape jelly elect use </s> <s> bring boil immediately reduce heat simmer cover 90 minute stir frequently </s> <s> add grape jelly optional last five minute cook stir melt </s> <s> need thin point add little additional chicken stock </s> <s> serve hot dog tangy mustard finely dice onion steam hot dog bun </s> <s> note use cheap grade hot dog grind one include chicken pork </s> <s> beef frank impart much flavor </s> <s> also prefer sauce without jelly personal taste issue decide last minute since jelly get add split sauce half use one tablespoon jelly one half </s>,green-bell-pepper ketchup prepared-yellow-mustard chili-powder worcestershire-sauce cider-vinegar allspice ground-cumin crushed-red-pepper-flakes ground-cloves <s> brown burger sausage cover pot </s> <s> break stir </s> <s> drain grease discard </s> <s> return drain burger sausage blend cook pot add ingredient except grape jelly elect use </s> <s> bring boil immediately reduce heat simmer cover 90 minute stir frequently </s> <s> add grape jelly optional last five minute cook stir melt </s> <s> need thin point add little additional chicken stock </s> <s> serve hot dog tangy mustard finely dice onion steam hot dog bun </s> <s> note use cheap grade hot dog grind one include chicken pork </s> <s> beef frank impart much flavor </s> <s> also prefer sauce without jelly personal taste issue decide last minute since jelly get add split sauce half use one tablespoon jelly one half </s>


Similarity:  0.6377310752868652
Recipe: 


...

,index,RecipeId,Name,RecipeCategory,RatingId,RecipeIngredientParts,RecipeInstructions,Recipe
119038,119038,174672,Crispy Soft Shell Crabs,< 60 Mins,5,soft-shelled-crabs milk all-purpose-flour ground-ginger paprika garlic-powder salted-butter mayonnaise,<s> soak crab milk 30 minute </s> <s> mix flour salt pepper ginger paprika garlic powder bag dredge crab mixture </s> <s> melt butter large skillet medium heat </s> <s> saute crab golden brown 5 minute per side </s> <s> spread bun mayonnaise top crab </s> <s> dollop coleslaw desire </s> <s> serve immediately </s>,soft-shelled-crabs milk all-purpose-flour ground-ginger paprika garlic-powder salted-butter mayonnaise <s> soak crab milk 30 minute </s> <s> mix flour salt pepper ginger paprika garlic powder bag dredge crab mixture </s> <s> melt butter large skillet medium heat </s> <s> saute crab golden brown 5 minute per side </s> <s> spread bun mayonnaise top crab </s> <s> dollop coleslaw desire </s> <s> serve immediately </s>


Similarity:  0.618255078792572
Recipe: 


,index,RecipeId,Name,RecipeCategory,RatingId,RecipeIngredientParts,RecipeInstructions,Recipe
181395,181395,286956,Pepperoni Pizza Chili,Beans,4,ground-beef kidney-beans stewed-tomatoes tomato-sauce water green-pepper salt mozzarella-cheese,<s> cook grind beef </s> <s> mix ingredient slow cooker </s> <s> cook 8 hour </s> <s> garnish mozzarella cheese desire </s>,ground-beef kidney-beans stewed-tomatoes tomato-sauce water green-pepper salt mozzarella-cheese <s> cook grind beef </s> <s> mix ingredient slow cooker </s> <s> cook 8 hour </s> <s> garnish mozzarella cheese desire </s>


Similarity:  0.6178948879241943
Recipe: 


,index,RecipeId,Name,RecipeCategory,RatingId,RecipeIngredientParts,RecipeInstructions,Recipe
201372,201372,326671,Sesame Pea Pods,Vegetable,3,snow-pea-pods sesame-seeds red-bell-peppers yellow-bell-pepper,<s> 10 inch skillet heat oil medium high heat </s> <s> add pea pod sesame seed cook 2 minute stir frequently pea pod crisp tender </s> <s> stir bell pepper </s> <s> cook 2 minute stir frequently bell pepper crisp tender </s>,snow-pea-pods sesame-seeds red-bell-peppers yellow-bell-pepper <s> 10 inch skillet heat oil medium high heat </s> <s> add pea pod sesame seed cook 2 minute stir frequently pea pod crisp tender </s> <s> stir bell pepper </s> <s> cook 2 minute stir frequently bell pepper crisp tender </s>


In [ ]:
# doc2vec model 2
inferred_vector2 = doc2vec_model2.infer_vector(data2[0].words)
sims2  = doc2vec_model2.docvecs.most_similar([inferred_vector2])

# print(inferred_vector)
print(sims2)

print("\nDocument")
print(data2[0])
print("Ingredients")
print(text_feature['RecipeIngredientParts'][0])
print("keywords",text_feature['Keywords'][0])

print("\nSimilar Documents")
for i in range(len(sims2)):
  print("\nKeywords", text_feature['Keywords'][sims2[i][0]])
  print("Similarity: ", sims2[i][1], d[sims2[i][0]])
  print("Ingredients", text_feature['RecipeIngredientParts'][sims2[i][0]])

## Doc 1 seems most similar to doc ..?
# print(' '.join(data[sims2[0][0]])+'\n')

[(0, 0.9417499303817749), (226530, 0.43687117099761963), (236331, 0.4289224147796631), (262923, 0.4241085350513458), (40363, 0.41844892501831055), (40362, 0.4128078818321228), (26358, 0.41044697165489197), (35661, 0.4048486053943634), (232112, 0.4039035439491272), (101843, 0.40330007672309875)]

Document
TaggedDocument(['toss', 'cup', 'berry', 'sugar', 'let', 'stand', 'minute', 'stir', 'occasionally', 'transfer', 'berry', 'sugar', 'mixture', 'food', 'processor', 'add', 'yogurt', 'process', 'smooth', 'strain', 'fine', 'sieve', 'pour', 'bake', 'pan', 'transfer', 'ice', 'cream', 'maker', 'process', 'accord', 'manufacturer', 'direction', 'freeze', 'uncover', 'edge', 'solid', 'centre', 'soft', 'transfer', 'processor', 'blend', 'smooth', 'return', 'pan', 'freeze', 'edge', 'solid', 'transfer', 'processor', 'blend', 'smooth', 'fold', 'remain', 'cup', 'blueberry', 'pour', 'plastic', 'mold', 'freeze', 'overnight', 'let', 'soften', 'slightly', 'serve'], [0])
Ingredients
['blueberries', 'granulate

**Doc2Vec Model 2:**

In [ ]:
KEYWORD = 'dessert'

vocab = list(doc2vec_model2.wv.vocab)
vocab[:1]
vocab.sort()
print('%d words in vocabulary...' % len(vocab))
print('Sample: %s   ...   %s...\n' % (vocab[:200], vocab[-200:]))

print('\nWords similar with "%s":' % KEYWORD)
pd.DataFrame(doc2vec_model2.wv.most_similar(KEYWORD))

10844 words in vocabulary...
Sample: ['aadd', 'aand', 'aash', 'abalone', 'abd', 'ability', 'abit', 'able', 'abm', 'aboout', 'aborbed', 'abot', 'abotu', 'abou', 'abour', 'about', 'abouts', 'abowl', 'abrasive', 'absinthe', 'absolut', 'absolute', 'absolutely', 'absorb', 'absorbent', 'absorded', 'absorption', 'abt', 'abundance', 'abundant', 'abundantly', 'aburage', 'abut', 'acc', 'accent', 'accept', 'acceptable', 'access', 'accessory', 'accident', 'accidentally', 'accidently', 'accommodate', 'accomodate', 'accompaniment', 'accompany', 'accomplish', 'accord', 'accordance', 'accordian', 'accordingly', 'accordion', 'accoring', 'account', 'accross', 'accumulate', 'accuracy', 'accurate', 'accurately', 'accustom', 'achar', 'ache', 'acheive', 'acheived', 'achieve', 'achiote', 'acid', 'acidic', 'acidity', 'acidulate', 'acinus', 'acne', 'acording', 'acorn', 'acquire', 'acrid', 'across', 'acrylic', 'act', 'action', 'activate', 'active', 'actively', 'activity', 'actual', 'actually', 'acv', 'adam', 'a

,0,1
0,dangerous,0.200079
1,brie,0.197726
2,obviously,0.192214
3,sonoma,0.191309
4,lea,0.190057
5,biga,0.189186
6,overboard,0.184965
7,cooktime,0.182425
8,genoa,0.180985
9,way,0.180765
